In [18]:
from utils import *
import numpy as np
import pickle
import networkx as nx
import neat
import argparse
from tqdm import tqdm
import re
from neat_monkey import *

In [3]:
def output_activation(x, current_balance):
    # scale the output to ensure that the sum of the bids does not exceed the available balance
    x = [max(bid, 0) for bid in x]
    scale_factor = 0
    if sum(x) != 0:
        scale_factor = current_balance / sum(x)
    return [bid * scale_factor for bid in x]

In [4]:
config_file = "monkey_config.o.cfg"
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

In [5]:
with open("winning_genome.o.pkl", "rb") as genome_file:
    winner = pickle.load(genome_file)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


In [6]:
#G.10.11
money = 744
model_input = construct_input("graph_info_s1/G.10.11.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

array([ 98.,   0.,   0., 247.,   0.,   0.,   0., 339.,  60.,   0.])

In [ ]:
y2 = np.array([140, 86, 15, 143, 78, 140, 101, 105, 86, 103])

x2 = construct_input("graph_info_s1/O.10.14.html", money)
max_degs = np.array(x2[0:len(x2)-2:3])
num_nodes = np.array(x2[2:len(x2):3])
tot_degs = np.array(x2[1:len(x2)-1:3])

pred(a, b, c, max_degs, num_nodes, tot_degs)

In [ ]:
#G.10.12
money = 734
model_input = construct_input("graph_info_s1/G.10.12.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

In [77]:
#O.10.13
money = 345
model_input = construct_input("graph_info_s1/O.10.13.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

array([  0.,   0., 189.,  34.,   0.,   0.,   0., 121.,   0.,   1.])

In [78]:
# O.10.14
money = 505
model_input = construct_input("graph_info_s1/O.10.14.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

array([  0.,   0., 196.,   0.,   0.,   0.,   0., 161.,  18., 130.])

In [ ]:
# J.20.15
money = 940
model_input = construct_input("graph_info_s1/J.20.15.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

In [ ]:
# J.20.16
money = 603
model_input = construct_input("graph_info_s1/J.20.16.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

In [ ]:
# J.30.17
money = 875
model_input = construct_input("graph_info_s1/J.30.17.html", money)
bids = np.array(output_activation(winner_net.activate(model_input), money)).round()
bids

In [ ]:
x = construct_input("graph_info_s1/O.10.13.html", money)
print(x)

In [ ]:
max_degs = np.array(x[0:len(x)-2:3])
num_nodes = np.array(x[2:len(x):3])
tot_degs = np.array(x[1:len(x)-1:3])

print(f'max degs:  {max_degs}')
print(f'num nodes: {num_nodes}')
print(f'tot degs:  {tot_degs}')

a = 5
b = 10

ovr = 1.0 * max_degs / np.sum(max_degs) + 1 * num_nodes / np.sum(num_nodes) + 20 * tot_degs / np.sum(tot_degs)
ovr = a * max_degs + b * num_nodes + tot_degs

res = (1000 / np.sum(ovr)) * ovr
print(res)


In [2]:
from datetime import datetime

def error(pred, y):
    return np.sum((y - pred) * (y - pred))

def pred(a, b, c, max, num, tot):
    res =  a * max + b * num + c * tot
    return (1000 / np.sum(res)) * res    

def pred_coefs(max, num, tot, y, a0=1, b0=1, c0=1):
    a = a0
    b = b0
    c = c0 # fix this at 1 bc normalization
    curr_err = error(pred(a, b, c, max, num, tot), y)

    abest = a
    bbest = b
    cbest = c
    errbest = curr_err

    start_time = datetime.now()

    while (datetime.now() - start_time).total_seconds() < 60:
        sets = [(a * 1.1, b, c),
                (a / 1.1, b, c),
                (a, b * 1.1, c),
                (a, b / 1.1, c)]
        
        curr_min = np.Inf

        for a_new, b_new, c_new in sets:
            est = pred(a_new, b_new, c_new, max, num, tot)

            if (est >= 0).all():
                new_err = error(est, y)

                if new_err < curr_min:
                    curr_min = new_err
                    a = a_new
                    b = b_new
                    c = c_new

                if new_err < errbest:
                    errbest = new_err
                    abest = a_new
                    bbest = b_new
                    cbest = c_new

    print(f'best solution is a={abest}, b={bbest}, c={cbest}, err={errbest}')
    return abest, bbest, cbest

In [4]:
y = np.array([100, 13, 132, 121, 94, 79, 102, 97, 148, 109])

x = construct_input("graph_info_s1/O.10.13.html", 1000)
max_degs = np.array(x[0:len(x)-2:3])
num_nodes = np.array(x[2:len(x):3])
tot_degs = np.array(x[1:len(x)-1:3])
print(x)
    

[71, 604.5, 63, 8, 44.0, 33, 149, 609.5, 48, 120, 511.5, 44, 108, 417.5, 46, 150, 276.0, 36, 80, 440.5, 45, 99, 534.5, 57, 68, 839.0, 59, 74, 728.0, 69, 1000]


In [7]:
pred(a, b, c, max_degs, num_nodes, tot_degs)

array([107.34722235,   8.74220232, 133.61960649, 110.48169723,
        93.47453885,  87.56892423,  87.49155634, 106.79409127,
       138.97452434, 125.50563659])

In [8]:
a, b, c = pred_coefs(max_degs, num_nodes, tot_degs, y, 1, 1, 1)

best solution is a=2.357947691000002, b=9.147537007594738e-14, c=1, err=919.3853188598862


In [ ]:
y = np.array([140, 86, 15, 143, 78, 140, 101, 105, 86, 103])

x2 = construct_input("graph_info_s1/O.10.14.html", 1000)
max_degs = np.array(x2[0:len(x2)-2:3])
num_nodes = np.array(x2[2:len(x2):3])
tot_degs = np.array(x2[1:len(x2)-1:3])

pred(a, b, c, max_degs, num_nodes, tot_degs)

In [17]:
money = 1000
model_input = construct_input("graph_info_s1/O.10.14.html", money)
max_degs = np.array(model_input[0:len(model_input)-2:3])
num_nodes = np.array(model_input[2:len(model_input):3])
tot_edges = np.array(model_input[1:len(model_input)-1:3])
print(max_degs, num_nodes, tot_edges)

ovr = tot_edges / num_nodes

res = (1000 / np.sum(ovr)) * ovr
print(res)

[ 53  55   9 158  93  55 113 123  41 106] [58 60 74 45 41 46 42 45 48 41] [744.5 471.  103.  587.  293.  588.  387.  430.5 377.5 385. ]
[140.92206817  86.18108481  15.28086028 143.20820037  78.45598213
 140.33364128 101.15887116 105.02747915  86.34118767 103.09062498]


In [66]:
def test_remainder(d1, d2, d3, money):
    total = d1 + d2 + d3
    print(f'total: {total}')

    metric = money - (money - total) * (money // (money - total))
    print(f'metric: {metric}')

    m1 = int(d1 / total * money)
    m2 = int(d2 / total * money)
    m3 = int(d3 / total * money)

    print(f'leftover: {money - m1 - m2 - m3}')

In [67]:
test_remainder(1, 2, 3, 1000)

total: 6
metric: 6
leftover: 1


In [76]:
test_set = np.array([1, 3, 2, 3, 2])
print(np.amax(test_set))
np.argwhere(test_set == np.amax(test_set)).flatten()

3


array([1, 3])